In [ ]:
import symforce
symforce.set_epsilon_to_symbol()

In [ ]:
import symforce.symbolic as sf
import numpy as np
from symforce.values import Values
from symforce import typing as T

In [ ]:
# 3 satellites, 3 poses
num_poses = 3
num_satellites = 3

In [ ]:
# Ground truth positions
x_pos = np.array([[3, 1, 0], [7, 4, 0], [11, 3, 0]])
s_pos = np.array([[-2, 7, 0], [13, 9, 0], [7, -6, 0]])

# Ground truth distances
ranges = np.zeros((num_satellites, num_poses))
for i in range(num_poses):
    for j in range(num_satellites):
        ranges[i,j] = np.linalg.norm(x_pos[i] - s_pos[j]) + np.random.normal(0, 0.1)

In [ ]:
# Assume all rotations = I

In [ ]:
# Store in values
initial_values = Values(
    world_T_body=[sf.Pose3.identity()] * num_poses,
    world_t_landmark=[sf.V3(s_pos[0]), sf.V3(s_pos[1]), sf.V3(s_pos[2])],
    odometry=[sf.Pose3(R = sf.Rot3(), t = sf.V3(4, 3, 0)), 
               sf.Pose3(R = sf.Rot3(), t = sf.V3(4, -1, 0))],
    ranges=ranges.tolist(),
    sigmas=sf.V6(1, 1, 1, 1, 1, 1),
    epsilon=sf.numeric_epsilon,
)

In [ ]:
def range_residual(
    pose: sf.Pose3, satellite: sf.V3, range: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    return sf.V1((pose.t - satellite).norm(epsilon=epsilon) - range)

In [ ]:
range_residual(sf.Pose3(R = sf.Rot3(), t = sf.V3(x_pos[0])), sf.V3(s_pos[0]), ranges[0,0], sf.numeric_epsilon)

In [ ]:
# def odometry_residual(
#     pose_a: sf.Pose2, pose_b: sf.Pose2, odom: sf.Pose2, epsilon: sf.Scalar
# ) -> sf.V3:
#     pose_diff = pose_b.compose(pose_a.inverse())
#     return T.cast(sf.V3, pose_diff.compose(odom.inverse()))

In [ ]:
def odometry_residual(
    world_T_a: sf.Pose3,
    world_T_b: sf.Pose3,
    a_T_b: sf.Pose3,
    diagonal_sigmas: sf.V6,
    epsilon: sf.Scalar,
) -> sf.V6:
    """
    Residual on the relative pose between two timesteps of the robot.
    Args:
        world_T_a: First pose in the world frame
        world_T_b: Second pose in the world frame
        a_T_b: Relative pose measurement between the poses
        diagonal_sigmas: Diagonal standard deviation of the tangent-space error
        epsilon: Small number for singularity handling
    """
    a_T_b_predicted = world_T_a.inverse() * world_T_b
    tangent_error = a_T_b_predicted.local_coordinates(a_T_b, epsilon=epsilon)
    return T.cast(sf.V6, sf.M.diag(diagonal_sigmas.to_flat_list()).inv() * sf.V6(tangent_error))

In [ ]:
pose_a = sf.Pose3(R = sf.Rot3(), t = sf.V3(x_pos[0]))
pose_b = sf.Pose3(R = sf.Rot3(), t = sf.V3(x_pos[1]))
a_b = sf.Pose3(R = sf.Rot3(), t = sf.V3(4, 3, 0))
odometry_residual(pose_a, pose_b, a_b, sf.V6(1, 1, 1, 1, 1, 1), sf.numeric_epsilon)

In [ ]:
from symforce.opt.factor import Factor

factors = []

# Range factors
for i in range(num_poses):
    for j in range(num_satellites):
        factors.append(Factor(
            residual=range_residual,
            keys=[f"world_T_body[{i}]", f"world_t_landmark[{j}]", f"ranges[{i}][{j}]", "epsilon"],
        ))

# Odometry factors
for i in range(num_poses - 1):
    factors.append(Factor(
        residual=odometry_residual,
        keys=[f"world_T_body[{i}]", f"world_T_body[{i + 1}]", f"odometry[{i}]", "sigmas", "epsilon"],
    ))

In [ ]:
from symforce.opt.optimizer import Optimizer

optimizer = Optimizer(
    factors=factors,
    optimized_keys=[f"world_T_body[{i}]" for i in range(num_poses)],
    # So that we save more information about each iteration, to visualize later:
    debug_stats=True,
)

In [ ]:
result = optimizer.optimize(initial_values)

In [ ]:
for i, pose in enumerate(result.optimized_values["world_T_body"]):
        print(f"world_T_body {i}: t = {pose.position()}, R = {pose.rotation().to_tangent()}")